# Capítulo 2

## Ejercicio 1
Descargar el Quijote https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79. Aplicar no solo count (para obtener el número de líneas) y show sino probar distintas sobrecargas del método show (con/sin truncate, indicando/sin indicar num de filas, etc) así como también los métodos, head, take, first (diferencias entre estos 3?).

### Inicio SparkSession

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

val spark = SparkSession 
 .builder
 .appName("Ejercicios")
 .getOrCreate()


Intitializing Scala interpreter ...

Spark Web UI available at http://L2111011.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1640613445028)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5cec1ca2


### Leo fichero

In [ ]:
val quijote = spark.read.format("csv")
 .option("header", "true")
 .option("inferSchema", "true")
 .load("C:/Users/sara.arribas/Documents/Big Data/Ejercicios/el_quijote.txt")

In [ ]:
quijote.show()

In [ ]:
quijote.count()

In [ ]:
quijote.show(5,false)

In [ ]:
quijote.head()

In [ ]:
quijote.take(5)

In [ ]:
quijote.first()

**Diferencia**: 
- head() coge la cabecera
- take(n) coge las n primeras líneas
- first() coge la primera línea (si hay cabecera la cabecera)

## Ejercicio 2

**Del ejercicio de M&M aplicar:**

**a) Otras operaciones de agregación como el Max con otro tipo de ordenamiento (descendiente).**

**b) hacer un ejercicio como el “where” de CA que aparece en el libro pero indicando más opciones de estados (p.e. NV, TX, CA, CO).**

**c) Hacer un ejercicio donde se calculen en una misma operación el Max, Min, Avg, Count. Revisar el API (documentación) donde encontrarán este ejemplo:**

**ds.agg(max($"age"), avg($"salary"))**
**ds.groupBy().agg(max($"age"), avg($"salary"))**


**d) Hacer también ejercicios en SQL creando tmpView**

In [ ]:
 val mnmDF = spark.read.format("csv")
 .option("header", "true")
 .option("inferSchema", "true") // más util para 
 .load("C:/Users/sara.arribas/Downloads/Ejemplos_Spark/mnm_dataset.csv")
 // Aggregate counts of all colors and groupBy() State and Color
 // orderBy() in descending order
 val maxMnMDF = mnmDF
 .select("State", "Color", "Count")
 .groupBy("State", "Color")
 .agg(max("Count").alias("Maximo"))
 .orderBy(desc("Maximo"))
 maxMnMDF.show()

 // Find the aggregate counts for California by filtering
 val caCountMnMDF = mnmDF
 .select("State", "Color", "Count")
 .where(col("State") === "CA" || col("State") === "NV" || col("State") === "TX" || col("State") === "CO")
 .groupBy("State", "Color")
 .agg(count("Count").alias("Total"))
 .orderBy(desc("Total"))
 // Show the resulting aggregations for California
 caCountMnMDF.show(10)


.

# Capítulo 3

**a) Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por defecto.**

In [ ]:
 val mnmDF = spark.read.format("csv")
 .option("header", "true")
 .option("inferSchema", "true") // más util para 
 .load("C:/Users/sara.arribas/Downloads/Ejemplos_Spark/mnm_dataset.csv")

In [ ]:
print(mnmDF.printSchema())

**b) Cuando se define un schema al definir un campo por ejemplo StructField('Delay',FloatType(), True) ¿qué significa el último parámetro Boolean?**

Significa si admite Null como parámetro.

**c). Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?**

Un dataframe es un dataset que al mismo tiempo está organizado también en columnas. Cada columna tiene su nombre correspondiente.

**d) Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos: JSON, CSV (dándole otro nombre para evitar sobrescribir el fichero origen), AVRO**




In [ ]:
val parquetPath = "C:/Users/sara.arribas/Downloads/Ejemplos_Spark/parquet/mnm_dataset.parquet"
mnmDF.write.format("parquet").save(parquetPath)

In [ ]:
mnmDF.write.json("/tmp/data/json/mnm_DF_json")

In [ ]:
mnmDF.write.format("csv").mode("overwrite").save("/tmp/data/csv/mnm_DF_csv")

In [ ]:
mnmDF.write.format("avro").mode("overwrite").save("/tmp/data/avro/mnm_DF_avro")

**f). Revisar al guardar los ficheros (p.e. json, csv, etc) el número de ficheros creados, revisar su contenido para comprender (constatar) como se guardan.**

Se guardan por particiones

**i) ¿A qué se debe que hayan más de un fichero?**

Porque se crean particiones al guardar el archivo

**ii) ¿Cómo obtener el número de particiones de un DataFrame?**


In [ ]:
mnmDF.rdd.partitions.size

**iii)  ¿Qué formas existen para modificar el número de particiones de un DataFrame?**

**iv) Llevar a cabo el ejemplo modificando el número de particiones a 1 y revisar de nuevo el/los ficheros guardados.**

In [ ]:
mnmDF.coalesce(4).write.format("json").mode("overwrite").save("/tmp/data/json/mnm_DF1_json")

En este caso al ser el archivo muy pequeño no pueden hacerse varias particiones pero el método es con coalesce(n) siendo n el número de particiones.

# Capítulo 4

**a) Realizar todos los ejercicios propuestos de libro** 

Hecho

**b) GlobalTempView vs TempView**

Una TempView está sujetaa a una sóla SparkSession, mientras que la GlobalTempView estará disponible siempre.

**c) Leer los AVRO, Parquet, JSON y CSV escritos en el cap3**

Hecho

# Capítulo 5

**a) Realizar todos los ejercicios propuestos de libro (excepto los de hive en caso de utilizar spark instalado en local y en el caso de RDBMS hacer únicamente ejemplo especificado más adelante)**

**b) Pros y Cons utilizar UDFs**

**c) Instalar MySQL, descargar driver y cargar datos de BBDD de empleados https://dev.mysql.com/doc/employee/en/**

i. Cargar con spark datos de empleados y departamentos

ii. Mediante Joins mostrar toda la información de los empleados además de su título y salario.
iii. Diferencia entre Rank y dense_rank (operaciones de ventana)
iv. Utilizando operaciones de ventana obtener el salario, posición (cargo)
y departamento actual de cada empleado, es decir, el último o más
reciente

In [3]:
val employeesDF = spark
 .read
 .format("jdbc")
 .option("url", "jdbc:mysql://localhost:3306/employees")
 .option("driver", "com.mysql.jdbc.Driver")
 .option("dbtable", "employees.sql")
 .option("user", "root")
 .option("password", "root")
 .load()

java.sql.SQLSyntaxErrorException:  Table 'employees.sql' doesn't exist